In [5]:
from kfp.v2.dsl import component, pipeline
from kfp.v2.dsl import Dataset, Output, Input, Metrics, Markdown, Artifact
from kfp.v2 import compiler

In [7]:
!export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:64

In [6]:
@component(
    base_image="gcr.io/ml-ops-segment-anything/sam:latest"
)
def batch_prediction(
    image_dir: str,
    prompt_json_file: str,
    visualization: Output[Markdown],
    
):
    import torch
    from typing import Dict, List
    from segment_anything import sam_model_registry, SamPredictor, SamAutomaticMaskGenerator
    import base64
    import numpy as np
    import cv2
    import logging
    from google.cloud import storage
    import base64
    import json
    import time
    
    storage_client = storage.Client()
    bucket = storage_client.bucket('sam-pipeline-test')
    # Get Model artifact from cloud storage
    blob = bucket.blob('model_artifacts/sam_vit_b_01ec64.pth')
    blob.download_to_filename('sam_vit_b_01ec64.pth')
    
    #Load the model
    sam = sam_model_registry["vit_b"](checkpoint="sam_vit_b_01ec64.pth")
    print(torch.cuda.is_available())
    sam.to("cuda")
    # Automatic mask generation
    mask_generator = SamAutomaticMaskGenerator(sam)
    # SAM Masking with prompts
    sam_predictor = SamPredictor(sam)
    
    # Initialize Images
    blobs = bucket.list_blobs(prefix=image_dir)
    image_extensions = ('.png', '.jpg', '.jpeg')
    image_blobs = [blob for blob in blobs if blob.name.lower().endswith(image_extensions)]
    
    #Prompt Json
    blob = bucket.blob(f'{image_dir}/{prompt_json_file}')
    prompt_json = json.loads(blob.download_as_text()) #returns prompts dictionary
    
    #Reshaping the image
    def reshape_image(image, size=256):
        # Ratio for showing up in Markdown
        if image.shape[0] < size and image.shape[1] < size: 
            ratio = 1
        else: 
            ratio = size / max(image.shape[0], image.shape[1])
        width = int(image.shape[1] * ratio)
        height = int(image.shape[0] * ratio)
        image = cv2.resize(image, (width, height))
        
        return image
    
    # Predict all images
    results = []
    for image_blob in image_blobs:
        image_bytes = image_blob.download_as_bytes()
        image_base64 = base64.b64encode(image_bytes).decode('utf-8')
        print(image_blob.name)
        jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
        image = cv2.cvtColor(cv2.imdecode(jpg_as_np, flags=1), cv2.COLOR_BGR2RGB)
        # Image resizing
        image = reshape_image(image)
        # Encoding it to base64 string
        image_base64 = base64.b64encode(cv2.imencode('.png', image)[1]).decode()
        
        # Predictions
        prediction = {}
        prediction["file_path"] = image_blob.name
        prediction["base64"] = image_base64
        prediction["image"] = image.tolist()
        prediction["masks"] = {}
        
        if image_blob.name in prompt_json: # Predicting with prompt inputs
            
            prompt_input = np.array(prompt_json[image_blob.name]).reshape(1,2)
            input_label = np.array([1])
            start = time.time()
            masks, scores, logits = sam_predictor.predict(
                                        point_coords=prompt_input,
                                        point_labels=input_label,
                                        multimask_output=False
                                        )
            print("Prediction time taken(with prompts): ", time.time() - start)
            h, w = masks.shape[-2:]
            masks = masks.reshape(h, w)
            prediction["masks"]['mask_1'] = masks
            prediction["prediction_type"] = "Predicting with Prompts"
            
        else: # Predicting without prompt inputs
            start = time.time()
            masks = mask_generator.generate(image)
            print("Prediction time taken(without prompts): ", time.time() - start)
            sorted_masks = sorted(masks, key=(lambda x: x['area']), reverse=True)
            for idx, mask in enumerate(sorted_masks):
            # TODO: Rewrite the result format, add more related scores and save it into a single json, with mask index
                prediction["masks"][f'mask_{idx}'] = mask['segmentation'].tolist()
            
            prediction["prediction_type"] = "Predicting without Prompts"
        
            
        results.append(prediction)
        torch.cuda.empty_cache()
    
    # TODO: Use Kubeflow Output to save json

    # TODO: Optimize the visualization
    with open(visualization.path, 'w') as f:
        alpha = 0.5
        for result in results:
            image = np.array(result["image"])
            f.write(f"## All Masks \n\n")
            f.write(f"# {result['file_path']} \n\n")
            f.write(f"# {result['prediction_type']} \n\n")
            f.write("<table><tr>")
            f.write(f'<td><img src="data:image/*;base64,{result["base64"]}" width=100% align="left"></td></tr>')
            for mask_name, mask in result["masks"].items():
                f.write(f"## {mask_name} \n\n")
                mask = np.array(mask)
                mask_color = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
                mask_color[mask == 0] = [0, 0, 0]
                mask_color[mask == 1] = [150, 0, 0]
                mask = np.array(mask)
                image[mask==1, :] = (1-alpha) * image[mask==1, :] + alpha * mask_color[mask==1, :3]
                image_base64 = base64.b64encode(cv2.imencode('.png', image)[1]).decode()
                f.write("<tr>")
                f.write(f'<td><img src="data:image/*;base64,{image_base64}" width=100% align="left"></td>')
                f.write("</tr>")
            f.write("</table>")

In [7]:
# Pipeline Initialization
@pipeline(
    pipeline_root="gs://sam-pipeline-test",
    name="sam-pipeline-test",
)
def sam_pipeline(
    image_dir: str = "batch_5"
):
    get_batch_prediction_op = (batch_prediction(image_dir=image_dir, prompt_json_file = "prompts_json_5.jsonl")
        .set_cpu_limit("8")
        .set_memory_limit("64G")
        .add_node_selector_constraint("cloud.google.com/gke-accelerator", "NVIDIA_TESLA_T4")
        .set_gpu_limit(1)
    )

In [8]:
compiler.Compiler().compile(
    pipeline_func=sam_pipeline,
    package_path='sam_pipe_test.json')



In [5]:
!gsutil cp sam_pipe_test.json gs://sam-pipeline-test

Copying file://sam_pipe_test.json [Content-Type=application/json]...
/ [1 files][  6.1 KiB/  6.1 KiB]                                                
Operation completed over 1 objects/6.1 KiB.                                      


In [9]:
from google.cloud import aiplatform

job = aiplatform.PipelineJob(display_name = 'sam_test-1',
                             template_path = 'sam_pipe_test.json',
                             enable_caching = False,
                             # failure_policy = "slow",
                             project="ml-ops-segment-anything",
                             location="us-west1",
                            )

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/633534855904/locations/us-west1/pipelineJobs/sam-pipeline-test-20230726161253
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/633534855904/locations/us-west1/pipelineJobs/sam-pipeline-test-20230726161253')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-west1/pipelines/runs/sam-pipeline-test-20230726161253?project=633534855904


In [6]:
!pip list | grep "kfp"

kfp                                    1.8.22
kfp-pipeline-spec                      0.1.16
kfp-server-api                         1.8.5


In [64]:
!pip install kfp==1.8.22

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.9/304.9 kB 11.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kfp: filename=kfp-1.8.22-py3-none-any.whl size=426971 sha256=76abdae182cf74a0ba6b45e327f7bbd65044fa748967d5865e8d6bcf5aa660bb
  Stored in directory: /home/jupyter/.cache/pip/wheels/74/c0/fc/bf0ab209fd6ae814d7efbc821076e948c3e4884f846583ab58
Successfully built kfp
  Attempting uninstall: kfp
    Found existing installation: kfp 1.8.20
    Uninstalling kfp-1.8.20:
      Successfully uninstalled kfp-1.8.20


In [17]:
!pip list | grep "kfp"

kfp                                    1.8.22
kfp-pipeline-spec                      0.1.16
kfp-server-api                         1.8.5


In [44]:
import numpy as np
from imantics import Polygons, Mask

# This can be any array
array = np.ones((100, 100))

polygons = Mask(array).polygons()

print(polygons.points)
print(polygons.segmentation)

[array([[ 0,  0],
       [ 0, 99],
       [99, 99],
       [99,  0]])]
[[0, 0, 0, 99, 99, 99, 99, 0]]


In [43]:
!pip install imantics

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.8 MB/s eta 0:00:0000:0100:01
  Created wheel for imantics: filename=imantics-0.1.12-py3-none-any.whl size=16015 sha256=734ccba4b3e2b9e3eb7b38e3e69eb4b18a41c9444275553093d02b1a35c98b93
  Stored in directory: /home/jupyter/.cache/pip/wheels/56/6a/be/4c60e88b14abec4e93234a1f7f91ce8abe1ae88a2b3eaad3ac
Successfully built imantics
